In [1]:
import tensorflow as tf
import numpy as np
import math
from tensorflow.python.ops import rnn, rnn_cell
import matplotlib.pyplot as plt

In [2]:
H = 256 # number of hidden layer neurons
D = 4 # input dimensionality
C = 2 # ouput classes 

n_input = 2 # MNIST data input (img shape: 2*2)
n_steps = 2 # timesteps

In [3]:
def autoencoder(input_shape=[None, D],
                n_filters=[1, 10, 10, 10],
                filter_sizes=[1, 1, 1, 1],
                corruption=False):
    """Build a deep denoising autoencoder w/ tied weights.
    Parameters
    ----------
    input_shape : list, optional
        Description
    n_filters : list, optional
        Description
    filter_sizes : list, optional
        Description
    Returns
    -------
    x : Tensor
        Input placeholder to the network
    z : Tensor
        Inner-most latent representation
    y : Tensor
        Output reconstruction of the input
    cost : Tensor
        Overall cost to use for training
    Raises
    ------
    ValueError
        Description
    """
    # %%
    # input to the network
    x = tf.placeholder(tf.float32, input_shape, name='x')


    # %%
    # ensure 2-d is converted to square tensor.
    if len(x.get_shape()) == 2:
        x_dim = np.sqrt(x.get_shape().as_list()[1])
        if x_dim != int(x_dim):
            raise ValueError('Unsupported input dimensions')
        x_dim = int(x_dim)
        x_tensor = tf.reshape(
            x, [-1, x_dim, x_dim, n_filters[0]])
    elif len(x.get_shape()) == 4:
        x_tensor = x
    else:
        raise ValueError('Unsupported input dimensions')
    current_input = x_tensor

    # %%
    # Optionally apply denoising autoencoder
    if corruption:
        current_input = corrupt(current_input)

    # %%
    # Build the encoder
    encoder = []
    shapes = []
    for layer_i, n_output in enumerate(n_filters[1:]):
        n_input = current_input.get_shape().as_list()[3]
        shapes.append(current_input.get_shape().as_list())
        W = tf.Variable(
            tf.random_uniform([
                filter_sizes[layer_i],
                filter_sizes[layer_i],
                n_input, n_output],
                -1.0 / math.sqrt(n_input),
                1.0 / math.sqrt(n_input)))
        b = tf.Variable(tf.zeros([n_output]))
        encoder.append(W)
        output = tf.nn.relu(
            tf.add(tf.nn.conv2d(
                current_input, W, strides=[1, 2, 2, 1], padding='SAME'), b))
        current_input = output

    # %%
    # store the latent representation
    z = current_input
    encoder.reverse()
    shapes.reverse()

    # %%
    # Build the decoder using the same weights
    for layer_i, shape in enumerate(shapes):
        W = encoder[layer_i]
        b = tf.Variable(tf.zeros([W.get_shape().as_list()[2]]))
        output = tf.nn.relu(tf.add(
            tf.nn.conv2d_transpose(
                current_input, W,
                tf.pack([tf.shape(x)[0], shape[1], shape[2], shape[3]]),
                strides=[1, 2, 2, 1], padding='SAME'), b))
        current_input = output

    # %%
    # now have the reconstruction through the network
    y = current_input
    # cost function measures pixel-wise difference
    cost = tf.reduce_sum(tf.square(y - x_tensor))

    # %%
    return {'x': x, 'z': z, 'y': y, 'cost': cost}

In [4]:
def test_mnist():
    """Test the convolutional autoencder using MNIST."""

    all_status = [1,2,3,4],[2,4,4,5],[5,4,8,6],[4,5,7,7],[5,6,0,8],[6,7,8,9],[7,8,9,10],[8,9,10,11],[9,10,11,12],[10,11,12,13]
    
    ae = autoencoder()

    # %%
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(ae['cost'])

    # %%
    # We create a session to use the graph
    sess = tf.Session()
    sess.run(tf.initialize_all_variables())

    # %%
    # Fit all training data
    #batch_size = 100
    n_epochs = 20000
    for epoch_i in range(n_epochs):
        sess.run(optimizer, feed_dict={ae['x']: all_status})
        print(epoch_i, sess.run(ae['cost'], feed_dict={ae['x']: all_status}))

    # %%
    # Plot example reconstructions
    n_examples = 10
    recon = sess.run(ae['y'], feed_dict={ae['x']: all_status})
    print(recon.shape)
    print recon
    '''
    fig, axs = plt.subplots(2, n_examples, figsize=(10, 2))
    for example_i in range(n_examples):
        axs[0][example_i].imshow(
            np.reshape(all_status[example_i, :], (4,1)))
        axs[1][example_i].imshow(recon)
        
    fig.show()
    plt.draw()
    plt.waitforbuttonpress()
    '''

In [5]:
if __name__ == '__main__':
    test_mnist()

(0, 2329.6582)
(1, 2326.8403)
(2, 2323.9426)
(3, 2320.9871)
(4, 2317.9868)
(5, 2314.8921)
(6, 2311.7144)
(7, 2308.4614)
(8, 2305.1379)
(9, 2301.7485)
(10, 2298.283)
(11, 2294.7363)
(12, 2291.1072)
(13, 2287.3989)
(14, 2283.605)
(15, 2279.7246)
(16, 2275.7544)
(17, 2271.7065)
(18, 2267.5415)
(19, 2263.2671)
(20, 2258.8496)
(21, 2254.3179)
(22, 2249.6736)
(23, 2244.917)
(24, 2240.0479)
(25, 2235.0659)
(26, 2229.9705)
(27, 2224.7607)
(28, 2219.4368)
(29, 2213.9978)
(30, 2208.4434)
(31, 2202.7732)
(32, 2196.9878)
(33, 2191.0874)
(34, 2185.0728)
(35, 2178.9446)
(36, 2172.7051)
(37, 2166.3557)
(38, 2159.8987)
(39, 2153.3374)
(40, 2146.6777)
(41, 2139.9473)
(42, 2133.1133)
(43, 2126.1831)
(44, 2119.187)
(45, 2112.1196)
(46, 2104.99)
(47, 2097.8081)
(48, 2090.5879)
(49, 2083.3462)
(50, 2076.0825)
(51, 2068.8071)
(52, 2061.5337)
(53, 2054.2771)
(54, 2047.054)
(55, 2039.8811)
(56, 2032.78)
(57, 2025.7633)
(58, 2018.8567)
(59, 2012.0803)
(60, 2005.4542)
(61, 1999.0011)
(62, 1992.7435)
(63, 1986.6